Apparently, we are lucky, and our predecessor already parsed out all addresses from god knows where. We can just fetch them with a scraper :)

In [3]:
link = "https://qvwx.de/sperrmuell-bonn-2024/"

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select
from webdriver_manager.firefox import GeckoDriverManager
import time

# Setup Selenium WebDriver with Firefox options
firefox_options = Options()
firefox_options.binary_location = "/usr/bin/firefox"  # Specify the path to the Firefox binary
# Uncomment the following line if you want to run Firefox in headless mode
firefox_options.add_argument("--headless")

driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)


WebDriverException: Message: <!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html><head>
<meta type="copyright" content="Copyright (C) 1996-2016 The Squid Software Foundation and contributors">
<meta http-equiv="Content-Type" CONTENT="text/html; charset=utf-8">
<title>ERROR: The requested URL could not be retrieved</title>
<style type="text/css"><!-- 
 /*
 * Copyright (C) 1996-2016 The Squid Software Foundation and contributors
 *
 * Squid software is distributed under GPLv2+ license and includes
 * contributions from numerous individuals and organizations.
 * Please see the COPYING and CONTRIBUTORS files for details.
 */

/*
 Stylesheet for Squid Error pages
 Adapted from design by Free CSS Templates
 http://www.freecsstemplates.org
 Released for free under a Creative Commons Attribution 2.5 License
*/

/* Page basics */
* {
	font-family: verdana, sans-serif;
}

html body {
	margin: 0;
	padding: 0;
	background: #efefef;
	font-size: 12px;
	co